In [2]:
import pandas as pd
import numpy as np

In [3]:
assumptions_raw = pd.read_excel("Mash3_BGS_Talk_v01.xlsx")

In [4]:
assumptions_raw.head(11)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,MN0,MN1,MN2,MN3,MN4,MN5,MN6,...,MN18,MN19,MN20,MN21,MN22,MN23,MN24,Unnamed: 29,YR1,YR2
0,ASSUMPTIONS,YR1,YR2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Starting Organic / Social Visitors,1000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Additive Social / Organic per Month,500,1000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,K-Factor (Users),0.2,0.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,K-Factor (Subscribers),0.5,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Traffic -> User Conversion Rate,0.25,0.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Cost of Paid Traffic,2,2.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Marketing Spend / Month,10000,25000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,MoM Retention (Users),0.3,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,User -> Subscriber Conversion Rate,0.1,0.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
assumptions = (
    assumptions_raw
    .iloc[1:,0:3]
    .head(10)
    .rename(columns=
            {'Unnamed: 0':'ASSUMPTIONS',
             'Unnamed: 1':'YR1',
             'Unnamed: 2': 'YR2'})
    )
assumptions

,ASSUMPTIONS,YR1,YR2
1,Starting Organic / Social Visitors,1000,NaN
2,Additive Social / Organic per Month,500,1000
3,K-Factor (Users),0.2,0.3
4,K-Factor (Subscribers),0.5,1
5,Traffic -> User Conversion Rate,0.25,0.3
6,Cost of Paid Traffic,2,2.5
7,Marketing Spend / Month,10000,25000
8,MoM Retention (Users),0.3,0.5
9,User -> Subscriber Conversion Rate,0.1,0.15
10,MoM Retention (Subscribers),0.4,0.6


In [6]:
subscribers = pd.DataFrame()
total_months = 25


In [7]:
def apply_assumption(row, assumption_name):
    if row['month'] < 13:
        return assumptions.loc[assumptions['ASSUMPTIONS'] == assumption_name, 'YR1'].iloc[0]
    else:
        return assumptions.loc[assumptions['ASSUMPTIONS'] == assumption_name, 'YR2'].iloc[0]

In [9]:
subscribers['month'] = [month for month in range(0,total_months)]

subscribers['Traffic -> User Conversion Rate'] = subscribers.apply(apply_assumption, axis=1, args=("Traffic -> User Conversion Rate",))
subscribers['K-Factor (Users)'] = subscribers.apply(apply_assumption, axis=1, args=("K-Factor (Users)",))
subscribers['Paid Traffic (Installs / Visitors)'] = subscribers.apply(apply_assumption, axis=1, args=("Cost of Paid Traffic",))
subscribers['Marketing Spend / Month'] = subscribers.apply(apply_assumption, axis=1, args=("Marketing Spend / Month",))
subscribers['MoM Retention (Users)'] = subscribers.apply(apply_assumption, axis=1, args=("MoM Retention (Users)",))  
subscribers['MoM Retention (Subscribers)'] = subscribers.apply(apply_assumption, axis=1, args=("MoM Retention (Subscribers)",))  
subscribers['K-Factor (Subscribers)'] = subscribers.apply(apply_assumption, axis=1, args=("K-Factor (Subscribers)",))
subscribers['User -> Subscriber Conversion Rate'] = subscribers.apply(apply_assumption, axis=1, args=("User -> Subscriber Conversion Rate",))



In [10]:
for i in subscribers['Month']:

    # ----------- Organic / Social Visitors --------------
    if i == 0:
        subscribers['Organic / Social Visitors'] = assumptions.loc[assumptions['ASSUMPTIONS'] == "Starting Organic / Social Visitors", 'YR1'].iloc[0]
    elif i < 13:
        subscribers.at[i, 'Organic / Social Visitors'] = subscribers.at[i-1, 'Organic / Social Visitors'] + assumptions.loc[assumptions['ASSUMPTIONS'] == "Additive Social / Organic per Month", 'YR1'].iloc[0]
    else:
        subscribers.at[i, 'Organic / Social Visitors'] = subscribers.at[i-1, 'Organic / Social Visitors'] + assumptions.loc[assumptions['ASSUMPTIONS'] == "Additive Social / Organic per Month", 'YR2'].iloc[0]
    
    subscribers['Cost / New User'] = subscribers['Paid Traffic (Installs / Visitors)'] / subscribers['Traffic -> User Conversion Rate']
    subscribers['New Acquired Users (Paid)'] = subscribers['Marketing Spend / Month'] / subscribers['Cost / New User']

    # ----------- Viral Acquired Visitors 
    if  i == 0:
        subscribers["Viral Acquired Visitors"] = 0
    else:
        subscribers.at[i,"Viral Acquired Visitors"] =  subscribers.at[i-1, '(All Channels) New Users'] * subscribers.at[i-1,'K-Factor (Users)'] + subscribers.at[i-1, '(All) Total Current Subscribers'] * subscribers.at[i-1,'K-Factor (Subscribers)']

    # ----------- New Organic + Viral Visitors
        
    subscribers['New Organic + Viral Visitors'] = (subscribers["Viral Acquired Visitors"] + subscribers['Organic / Social Visitors']) * subscribers['Traffic -> User Conversion Rate']

    subscribers['(All Channels) New Users'] = subscribers['New Acquired Users (Paid)'] + subscribers['New Organic + Viral Visitors']

    if  i == 0:
        subscribers["(All Channels) Retained Users"] = 0
    else:
        subscribers.at[i,"(All Channels) Retained Users"] =  subscribers.at[i-1, "(All Channels) (New + Retained) Users"] * subscribers.at[i-1,'MoM Retention (Users)'] - subscribers.at[i-1, "(New) Subscribers"] 

    if  i == 0:
        subscribers["(All Channels) (New + Retained) Users"] = subscribers['(All Channels) New Users'] + subscribers["(All Channels) Retained Users"]
    else:    
       subscribers.at[i,"(All Channels) (New + Retained) Users"] = subscribers.at[i,'(All Channels) New Users'] + subscribers.at[i,"(All Channels) Retained Users"] - subscribers.at[i-1, "(New) Subscribers"]

    subscribers['(New) Subscribers'] = subscribers['User -> Subscriber Conversion Rate'] * subscribers["(All Channels) (New + Retained) Users"]

    if  i == 0:
        subscribers["(Retained) Subscribers"] = 0
    else:
        subscribers.at[i,"(Retained) Subscribers"] =  subscribers.at[i-1, "(New) Subscribers"] * subscribers.at[i-1,'MoM Retention (Subscribers)']

    subscribers["(All) Total Current Subscribers"] = subscribers["(Retained) Subscribers"] + subscribers['(New) Subscribers']

    subscribers['(All Channels) Cumulative New Users'] = subscribers['(All Channels) New Users'].cumsum()
    

C:\Users\usuario\AppData\Local\Temp\ipykernel_19864\194550798.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '467.1875' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  subscribers.at[i,"Viral Acquired Visitors"] =  subscribers.at[i-1, '(All Channels) New Users'] * subscribers.at[i-1,'K-Factor (Users)'] + subscribers.at[i-1, '(All) Total Current Subscribers'] * subscribers.at[i-1,'K-Factor (Subscribers)']
C:\Users\usuario\AppData\Local\Temp\ipykernel_19864\194550798.py:29: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '373.75' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  subscribers.at[i,"(All Channels) Retained Users"] =  subscribers.at[i-1, "(All Channels) (New + Retained) Users"] * subscribers.at[i-1,'MoM Retention (Users)'] - subscr

In [11]:
subscribers.iloc[:,10:16].head(15)

,Organic / Social Visitors,Cost / New User,New Acquired Users (Paid),Viral Acquired Visitors,New Organic + Viral Visitors,(All Channels) New Users
0,1000,8.000000,1250.0,0.000000,250.000000,1500.000000
1,1500,8.000000,1250.0,375.000000,468.750000,1718.750000
2,2000,8.000000,1250.0,467.187500,616.796875,1866.796875
3,2500,8.000000,1250.0,513.417969,753.354492,2003.354492
4,3000,8.000000,1250.0,552.180420,888.045105,2138.045105
5,3500,8.000000,1250.0,589.729318,1022.432330,2272.432330
6,4000,8.000000,1250.0,627.081014,1156.770254,2406.770254
7,4500,8.000000,1250.0,664.400665,1291.100166,2541.100166
8,5000,8.000000,1250.0,701.715108,1425.428777,2675.428777
9,5500,8.000000,1250.0,739.028705,1559.757176,2809.757176


In [12]:
subscribers['(All) Total Current Subscribers']

0      150.000000
1      246.875000
2      280.117188
3      303.019043
4      324.240594
5      345.189097
6      366.093228
7      386.990150
8      407.885899
9      428.781459
10     449.676987
11     470.572510
12     491.468033
13    1045.394854
14    1701.156040
15    1991.976045
16    2167.622380
17    2301.081282
18    2419.370831
19    2532.198960
20    2643.060989
21    2753.215222
22    2863.114648
23    2972.922344
24    3082.697017
Name: (All) Total Current Subscribers, dtype: float64

In [13]:
import plotly.express as px

In [14]:
px.line(subscribers,
        x = 'month',
        y = "(All) Total Current Subscribers")

In [15]:
df2 = pd.read_excel('Mash3_BGS_Talk_v02b.xlsx')

In [16]:
df2.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,MN0,MN1,MN2,MN3,MN4,MN5,MN6,...,MN18,MN19,MN20,MN21,MN22,MN23,MN24,Unnamed: 29,YR1,YR2
0,ASSUMPTIONS,YR1,YR2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Traffic Sources,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Starting Organic / Social Visitors,1000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Additive Social / Organic per Month,500,1000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Cost of Paid Traffic,2,2.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
df_clean = (
    df2
    .iloc[1:,0:3]
    .head(13)
    .rename(columns=
            {'Unnamed: 0':'ASSUMPTIONS',
             'Unnamed: 1':'YR1',
             'Unnamed: 2': 'YR2'})
    .dropna(subset=['YR1'])
    )
df_clean.head(15)

,ASSUMPTIONS,YR1,YR2
2,Starting Organic / Social Visitors,1000,NaN
3,Additive Social / Organic per Month,500,1000
4,Cost of Paid Traffic,2,2.5
5,Marketing Spend / Month,10000,25000
6,K-Factor (Users),0.2,0.3
7,K-Factor (Subscribers),0.5,1
10,Traffic -> User Conversion Rate,0.25,0.3
11,MoM Retention (Users),0.3,0.5
12,User -> Subscriber Conversion Rate,0.1,0.15
13,MoM Retention (Subscribers),0.4,0.6


In [18]:
px.line(
    subscribers,
    x = 'Month',
    y = "(All) Total Current Subscribers",
    title = "Evolution of Subscribers over Time",
    labels = {"(All) Total Current Subscribers" : "Total Subscribers"}
    ).update_layout(
        title_font = dict(size = 35),
        title = {'x':0.5,
                 'y': .94,
                 'xanchor': 'center'} ,
        height = 550
    ).update_xaxes(
        title_font = dict(
            size = 20
        ),
        tickfont_size= 20
    ).update_yaxes(
        title_font = dict(
            size = 20
        ),
        tickfont_size= 15
    ).add_annotation(
        text = "Change in slope due to increase in<br><b>conversion</b> and <b>retention</b>",
        x = 12,
        y = 491.468,
        arrowhead = 1,
        yshift=10,
        xanchor="right",
        font=dict(
            size=17
            ),
        arrowsize=2,
        align="center",       
    )

In [19]:
fig = px.line(
    subscribers,
    x='Month',
    y="(All) Total Current Subscribers",
    title="Evolution of Subscribers over Time",
    labels={"(All) Total Current Subscribers": "Total Subscribers"}
)

fig.update_layout(
    plot_bgcolor='white',  
    paper_bgcolor='white',
    title_font=dict(size=35),
    title={'x':0.5, 'y':0.96, 'xanchor': 'center'},
    height=800,  
    width=1200, 
).update_xaxes(
    title_font=dict(size=20),
    tickfont_size=20,
    showgrid=False 
).update_yaxes(
    title_font=dict(size=20),
    tickfont_size=15,
    showgrid=False
).add_annotation(
    text="Change in slope due to increase in<br><b>conversion</b>, <b>retention</b> and <b>ad spend</b>",
    x=12,
    y=491.468,
    arrowhead=1,
    yshift=10,
    xanchor="right",
    showarrow=True,
    font=dict(size=20),
    arrowsize=2,
    align="center",
    ax=-50,  
    ay=-30
)

fig.add_annotation(
    text="Projected subscriber growth over the first two years based on business model simulations,<br>with product improvements leading to increased retention and conversion rates.",
    xref="paper", yref="paper",
    x=0.5, y=0.96,  
    showarrow=False,
    font=dict(size=20, color="grey"),
    xanchor='center', yanchor='bottom',
)

fig.update_layout(template='plotly_white') 


fig.show()

In [20]:

def subscription_growth(time_frame, assumptions, retention_subscribers = 0):

    subscribers = pd.DataFrame()

    subscribers['month'] = range(0,time_frame)

    if retention_subscribers == 0:
        subscribers['MoM Retention (Subscribers)'] = subscribers.apply(apply_assumption, axis=1, args=("MoM Retention (Subscribers)",))  
    else:
        subscribers['MoM Retention (Subscribers)'] = retention_subscribers

    subscribers['Traffic -> User Conversion Rate'] = subscribers.apply(apply_assumption, axis=1, args=("Traffic -> User Conversion Rate",))
    subscribers['K-Factor (Users)'] = subscribers.apply(apply_assumption, axis=1, args=("K-Factor (Users)",))
    subscribers['Paid Traffic (Installs / Visitors)'] = subscribers.apply(apply_assumption, axis=1, args=("Cost of Paid Traffic",))
    subscribers['Marketing Spend / Month'] = subscribers.apply(apply_assumption, axis=1, args=("Marketing Spend / Month",))
    subscribers['MoM Retention (Users)'] = subscribers.apply(apply_assumption, axis=1, args=("MoM Retention (Users)",))  
    subscribers['K-Factor (Subscribers)'] = subscribers.apply(apply_assumption, axis=1, args=("K-Factor (Subscribers)",))
    subscribers['User -> Subscriber Conversion Rate'] = subscribers.apply(apply_assumption, axis=1, args=("User -> Subscriber Conversion Rate",))

    for i in subscribers['month']:

        # ----------- Organic / Social Visitors --------------
        if i == 0:
            subscribers['Organic / Social Visitors'] = assumptions.loc[assumptions['ASSUMPTIONS'] == "Starting Organic / Social Visitors", 'YR1'].iloc[0]
        elif i < 13:
            subscribers.at[i, 'Organic / Social Visitors'] = subscribers.at[i-1, 'Organic / Social Visitors'] + assumptions.loc[assumptions['ASSUMPTIONS'] == "Additive Social / Organic per Month", 'YR1'].iloc[0]
        else:
            subscribers.at[i, 'Organic / Social Visitors'] = subscribers.at[i-1, 'Organic / Social Visitors'] + assumptions.loc[assumptions['ASSUMPTIONS'] == "Additive Social / Organic per Month", 'YR2'].iloc[0]
        
        subscribers['Cost / New User'] = subscribers['Paid Traffic (Installs / Visitors)'] / subscribers['Traffic -> User Conversion Rate']
        subscribers['New Acquired Users (Paid)'] = subscribers['Marketing Spend / Month'] / subscribers['Cost / New User']

        # ----------- Viral Acquired Visitors 
        if  i == 0:
            subscribers["Viral Acquired Visitors"] = 0
        else:
            subscribers.at[i,"Viral Acquired Visitors"] =  subscribers.at[i-1, '(All Channels) New Users'] * subscribers.at[i-1,'K-Factor (Users)'] + subscribers.at[i-1, '(All) Total Current Subscribers'] * subscribers.at[i-1,'K-Factor (Subscribers)']

        # ----------- New Organic + Viral Visitors
            
        subscribers['New Organic + Viral Visitors'] = (subscribers["Viral Acquired Visitors"] + subscribers['Organic / Social Visitors']) * subscribers['Traffic -> User Conversion Rate']

        subscribers['(All Channels) New Users'] = subscribers['New Acquired Users (Paid)'] + subscribers['New Organic + Viral Visitors']

        if  i == 0:
            subscribers["(All Channels) Retained Users"] = 0
        else:
            subscribers.at[i,"(All Channels) Retained Users"] =  subscribers.at[i-1, "(All Channels) (New + Retained) Users"] * subscribers.at[i-1,'MoM Retention (Users)'] - subscribers.at[i-1, "(New) Subscribers"] 

        if  i == 0:
            subscribers["(All Channels) (New + Retained) Users"] = subscribers['(All Channels) New Users'] + subscribers["(All Channels) Retained Users"]
        else:    
            subscribers.at[i,"(All Channels) (New + Retained) Users"] = subscribers.at[i,'(All Channels) New Users'] + subscribers.at[i,"(All Channels) Retained Users"] - subscribers.at[i-1, "(New) Subscribers"]

        subscribers['(New) Subscribers'] = subscribers['User -> Subscriber Conversion Rate'] * subscribers["(All Channels) (New + Retained) Users"]

        if  i == 0:
            subscribers["(Retained) Subscribers"] = 0
        else:
            subscribers.at[i,"(Retained) Subscribers"] =  subscribers.at[i-1, "(New) Subscribers"] * subscribers.at[i-1,'MoM Retention (Subscribers)']

        subscribers["(All) Total Current Subscribers"] = subscribers["(Retained) Subscribers"] + subscribers['(New) Subscribers']

        subscribers['(All Channels) Cumulative New Users'] = subscribers['(All Channels) New Users'].cumsum()

    return subscribers

In [21]:
subscription25 = subscription_growth(time_frame = 24, assumptions = assumptions, retention_subscribers = 0.25)
subscription65 = subscription_growth(time_frame = 24, assumptions = assumptions, retention_subscribers = 0.65)
subscription65.head()

C:\Users\usuario\AppData\Local\Temp\ipykernel_19864\2869945529.py:60: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '37.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

C:\Users\usuario\AppData\Local\Temp\ipykernel_19864\2869945529.py:37: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '455.9375' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

C:\Users\usuario\AppData\Local\Temp\ipykernel_19864\2869945529.py:48: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '373.75' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

C:\Users\usuario\AppData\Local\Temp\ipykernel_19864\2869945529.py:60: FutureWarning:

Setting an item of incompa

,month,MoM Retention (Subscribers),Traffic -> User Conversion Rate,K-Factor (Users),Paid Traffic (Installs / Visitors),Marketing Spend / Month,MoM Retention (Users),K-Factor (Subscribers),User -> Subscriber Conversion Rate,Organic / Social Visitors,...,New Acquired Users (Paid),Viral Acquired Visitors,New Organic + Viral Visitors,(All Channels) New Users,(All Channels) Retained Users,(All Channels) (New + Retained) Users,(New) Subscribers,(Retained) Subscribers,(All) Total Current Subscribers,(All Channels) Cumulative New Users
0,0,0.65,0.25,0.2,2.0,10000,0.3,0.5,0.1,1000,...,1250.0,0.000000,250.000000,1500.000000,0.000000,1500.000000,150.000000,0.000000,150.000000,1500.000000
1,1,0.65,0.25,0.2,2.0,10000,0.3,0.5,0.1,1500,...,1250.0,375.000000,468.750000,1718.750000,300.000000,1868.750000,186.875000,97.500000,284.375000,3218.750000
2,2,0.65,0.25,0.2,2.0,10000,0.3,0.5,0.1,2000,...,1250.0,485.937500,621.484375,1871.484375,373.750000,2058.359375,205.835938,121.468750,327.304688,5090.234375
3,3,0.65,0.25,0.2,2.0,10000,0.3,0.5,0.1,2500,...,1250.0,537.949219,759.487305,2009.487305,411.671875,2215.323242,221.532324,133.793359,355.325684,7099.721680
4,4,0.65,0.25,0.2,2.0,10000,0.3,0.5,0.1,3000,...,1250.0,579.560303,894.890076,2144.890076,443.064648,2366.422400,236.642240,143.996011,380.638251,9244.611755


In [22]:
import plotly.graph_objects as go


fig = px.line(
    subscribers,
    x='month',
    y="(All) Total Current Subscribers",
    title="Evolution of Subscribers over Time",
    labels={
        "(All) Total Current Subscribers": "Total Subscribers",
        "month": "Month"}
)

fig.update_layout(
    plot_bgcolor='white',  
    paper_bgcolor='white',
    title_font=dict(size=35),
    title={'x':0.5, 'y':0.96, 'xanchor': 'center'},
    height=800,  
    width=1200,
    showlegend=False 
).update_xaxes(
    title_font=dict(size=20),
    tickfont_size=20,
    showgrid=False 
).update_yaxes(
    title_font=dict(size=20),
    tickfont_size=15,
    showgrid=False
).add_annotation(
    text="Change in slope due to increase in<br><b>conversion</b>, <b>retention</b> and <b>ad spend</b>",
    x=12,
    y=491.468,
    arrowhead=1,
    yshift=10,
    xanchor="right",
    showarrow=True,
    font=dict(size=20),
    arrowsize=2,
    align="center",
    ax=-50,  
    ay=-30
)

fig.add_annotation(
    text="Projected subscriber growth over the first two years based on business model simulations,<br>with product improvements leading to increased retention and conversion rates.",
    xref="paper", yref="paper",
    x=0.5, y=0.96,  
    showarrow=False,
    font=dict(size=20, color="grey"),
    xanchor='center', yanchor='bottom',
)

#fig.update_layout(template='plotly_white') 

# Agrega la línea para el límite inferior de la incertidumbre
fig.add_trace(go.Scatter(
    x=subscription25['month'],
    y=subscription25['(All) Total Current Subscribers'],
    mode='lines',
    line=dict(width=0),
    showlegend=False
))

# Agrega la línea para el límite superior de la incertidumbre
fig.add_trace(go.Scatter(
    x=subscription65['month'],
    y=subscription65['(All) Total Current Subscribers'],
    mode='lines',
    fill='tonexty',  # Esto rellena el área entre la línea superior y la línea inferior
    fillcolor='rgba(173, 216, 230, 0.5)',  # Un color azul claro con transparencia
    line=dict(width=0),
    showlegend=False
))

fig.add_annotation(
    text="The area represent all the posible growth<br>"
         "based on the different retentions present<br>"
         "in the industry: <b>25 %</b> to <b>65 %</b> based on research.",
    x=18,
    y=2500,
    arrowhead=1,
    yshift=10,
    xanchor="right",
    showarrow=True,
    font=dict(size=20),
    arrowsize=2,
    align="center",
    ax=-50,  
    ay=-30
)

fig.show()

In [23]:
subscribers.to_csv('subscription_simulation_data.csv')

In [47]:

fig2 = px.scatter(
    subscribers,
    x='Marketing Spend / Month', 
    y='(New) Subscribers', 
    trendline='ols',  
    title='Marketing Spend vs. Subscriber Growth'
)
fig2.update_layout(
    plot_bgcolor='white',  
    paper_bgcolor='white',
    title_font=dict(size=35),
    title={'x':0.5, 'y':0.96, 'xanchor': 'center'},
    height=800,  
    width=1200,
    showlegend=False 
)

fig2.add_annotation(
    text="Expected subscriber growth based on proyected marketing spend year over year.",
    xref="paper", yref="paper",
    x=0.5, y=1.02,  
    showarrow=False,
    font=dict(size=20, color="grey"),
    xanchor='center', yanchor='bottom',
)

fig2.add_annotation(
    text="While the exact relationship between <b>marketing spend</b> and<br>"
         "its influence on subscriber growth is not fully known,<br>"
         "marketing spend is expected to be one of the main factors<br>"
         "affecting the <b>volume of users</b>.",
    x=22000,
    y=1500,
    arrowhead=1,
    yshift=10,
    xanchor="right",
    showarrow=False,
    font=dict(size=20),
    arrowsize=2,
    align="center",
    ax=-50,  
    ay=-30
)

fig2.update_layout(
    xaxis_title='Marketing Spend per Month',
    yaxis_title='Total Subscribers',
    title_font=dict(size=24),
    xaxis_tickfont_size=16,
    yaxis_tickfont_size=16
)


fig2.show()